In [58]:
import pandas as pd
import xgboost as xgb 
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

**Read Model Data**

In [59]:
X_train=pd.read_csv("./Model_Data/XGBOOST_X_train.csv")
X_test=pd.read_csv("./Model_Data/XGBOOST_X_test.csv")
y_train=pd.read_csv("./Model_Data/XGBOOST_y_train.csv")
y_test=pd.read_csv("./Model_Data/XGBOOST_y_test.csv")

In [60]:
X_train = X_train.drop(X_train.columns[0], axis=1)
X_test = X_test.drop(X_test.columns[0], axis=1)
y_train = y_train.drop(y_train.columns[0], axis=1)
y_test = y_test.drop(y_test.columns[0], axis=1)

**Set Model Parameters**

In [61]:
parameters={'max_depth': 6, 
            'eta' : 0.3,
            'objective':'binary:logistic',
            'seed' : 4241,
            'min_child_weight' : 5,
            'n_estimators' : 250
           }

**Train Model**

In [62]:
n_first = int(X_train.shape[0]*0.8)
model = xgb.XGBClassifier(**parameters, nthread = 1)
model = model.fit(X_train.iloc[:n_first], y_train[:n_first],
          eval_set = [(X_train.iloc[:n_first], y_train[:n_first]),
                      (X_train.iloc[n_first:], y_train[n_first:])],
          eval_metric = 'auc',
          verbose = True,
          early_stopping_rounds=20) 

[0]	validation_0-auc:0.88369	validation_1-auc:0.88564
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.89466	validation_1-auc:0.89522
[2]	validation_0-auc:0.90180	validation_1-auc:0.90209
[3]	validation_0-auc:0.90494	validation_1-auc:0.90565
[4]	validation_0-auc:0.90920	validation_1-auc:0.90939
[5]	validation_0-auc:0.91098	validation_1-auc:0.91084
[6]	validation_0-auc:0.91308	validation_1-auc:0.91316
[7]	validation_0-auc:0.91497	validation_1-auc:0.91489
[8]	validation_0-auc:0.91665	validation_1-auc:0.91582
[9]	validation_0-auc:0.91842	validation_1-auc:0.91801
[10]	validation_0-auc:0.91998	validation_1-auc:0.91900
[11]	validation_0-auc:0.92168	validation_1-auc:0.91980
[12]	validation_0-auc:0.92265	validation_1-auc:0.92032
[13]	validation_0-auc:0.92339	validation_1-auc:0.92087
[14]	validation_0-auc:0.92457	validation_1-auc:0.92158
[15]	validation_0-auc:0.9255